Bibliotecas

In [ ]:
import cv2
import pandas as pd
import numpy as np
import random

Funções

In [ ]:
def find_parent_contours(bordas, min_contour_area):
    contours, _ = cv2.findContours(bordas, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    parent_contours = [contour for contour in contours if cv2.contourArea(contour) > min_contour_area]
    return pd.DataFrame({'contour': parent_contours})

def generate_random_lines(num_lines, max_line_length, max_angle_deviation, max_rho_deviation, image_shape):
    max_rho = np.hypot(image_shape[0], image_shape[1])
    lines = [(random.uniform(0, max_rho), random.uniform(0, np.pi)) for _ in range(num_lines)]
    return add_randomness_to_lines(lines, max_angle_deviation, max_rho_deviation)

def add_randomness_to_lines(lines, max_angle_deviation, max_rho_deviation):
    return [(rho + random.uniform(-max_rho_deviation, max_rho_deviation),
             theta + random.uniform(-max_angle_deviation, max_angle_deviation)) 
            for rho, theta in lines]

def check_points_in_lines(contour_points, lines, tolerance):
    def point_in_line(point, rho, theta, tolerance):
        x, y = point
        a, b = np.cos(theta), np.sin(theta)
        dist = abs(a * x + b * y - rho) / np.hypot(a, b)
        return dist < tolerance

    pontos_contornos = [(x, y) for [x, y] in contour_points 
                        if any(point_in_line((x, y), rho, theta, tolerance) for rho, theta in lines)]
    return pontos_contornos

def fator_conversao(regua_micro, regua_macro, resolucao_ppi):
    return (regua_micro / regua_macro) * 2.54 / resolucao_ppi


Carregamento da Imagem

In [ ]:
import cv2

def carregar_imagem(caminho_da_imagem):
    return cv2.imread(caminho_da_imagem, cv2.IMREAD_COLOR)

def converter_para_cinza(imagem):
    return cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)

def aplicar_filtro_gaussiano(imagem_cinza, tamanho_kernel=(5, 5), desvio_padrao=0):
    return cv2.GaussianBlur(imagem_cinza, tamanho_kernel, desvio_padrao)

# Caminho da imagem
caminho_da_imagem = '~/Imagens/Teste/mapa-do-brasil-estados-branco-sem-legenda.jpg'

# Carregar a imagem
imagem = carregar_imagem(caminho_da_imagem)

# Converter a imagem para escala de cinza
imagem_cinza = converter_para_cinza(imagem)

# Aplicar um filtro Gaussiano para redução de ruído
imagem_filtrada = aplicar_filtro_gaussiano(imagem_cinza)


Aplicação do Método de Canny

In [ ]:
import cv2

def aplicar_canny(imagem, limite_inferior, limite_superior):
    return cv2.Canny(imagem, limite_inferior, limite_superior)

# Parâmetros do método de Canny
limite_inferior = 50
limite_superior = 50

# Aplicar o método de Canny para detecção de bordas
bordas = aplicar_canny(imagem_filtrada, limite_inferior, limite_superior)


Visualização das Bordas

In [ ]:
import matplotlib.pyplot as plt

def exibir_imagem_bordas(bordas, titulo='Bordas detectadas com o método de Canny'):
    plt.imshow(bordas, cmap='gray')
    plt.title(titulo)
    plt.axis('off')
    plt.show()

# Exibir a imagem com as bordas detectadas
exibir_imagem_bordas(bordas)


Criação das Retas

In [ ]:
import cv2
import numpy as np
import pandas as pd

def processar_imagem(imagem_filtrada, bordas, min_contour_area=10, max_random_lines=40, max_line_length=500,
                     max_angle_deviation=np.pi / 2, max_rho_deviation=100, tolerance=5):
    try:
        if imagem_filtrada is None:
            raise ValueError("Não foi possível ler a imagem. Verifique o caminho da imagem.")

        # Encontrar os contornos relevantes na imagem
        parent_contours_df = find_parent_contours(bordas, min_contour_area)

        if len(parent_contours_df) >= 2:
            # Gerar linhas aleatórias
            randomized_lines = generate_random_lines(max_random_lines, max_line_length, max_angle_deviation, max_rho_deviation, imagem_filtrada.shape)

            # Desenhar as linhas detectadas com aleatoriedade
            for rho, theta in randomized_lines:
                a, b = np.cos(theta), np.sin(theta)
                x0, y0 = a * rho, b * rho
                x1, y1 = int(x0 + max_line_length * (-b)), int(y0 + max_line_length * (a))
                x2, y2 = int(x0 - max_line_length * (-b)), int(y0 - max_line_length * (a))
                cv2.line(imagem_filtrada, (x1, y1), (x2, y2), (0, 0, 255), 2)

            # Verificar se os pontos dos contornos estão contidos nas retas
            for _, row in parent_contours_df.iterrows():
                contour_points = row['contour']
                check_points_in_lines(contour_points, randomized_lines, tolerance)
        else:
            print("Não foram encontrados contornos suficientes na imagem.")

    except ValueError as ve:
        print("Erro ao processar a imagem:", ve)
    except Exception as e:
        print("Ocorreu um erro inesperado:", e)

# Chamada da função de processamento
processar_imagem(imagem_filtrada, bordas)


Visualização das Imagens com as retas

In [ ]:
import cv2
from google.colab.patches import cv2_imshow  # Import necessário para usar cv2_imshow no Google Colab

def exibir_imagem(imagem):
    cv2_imshow(imagem)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Exibir a imagem com as linhas traçadas
exibir_imagem(imagem_filtrada)


Vizualização pelo imageshow do matplotlib

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import cv2

def exibir_imagem(imagem):
    plt.imshow(imagem)
    plt.axis('off')
    plt.show()

def carregar_imagem(caminho_da_imagem):
    imagem = Image.open(caminho_da_imagem)
    return imagem

# Modifique o carregamento da imagem
caminho_da_imagem = 'data/sand-grains-under-microscope-gary-greenberg-1.jpg'
imagem = carregar_imagem(caminho_da_imagem)

# Exibir a imagem
exibir_imagem(imagem)
